In [1]:
import requests

API_URL = "http://localhost:5433"
MODEL_NAME = "scb10x/typhoon2.1-gemma3-4b:latest"

def generate(prompt: str) -> str:
    """เรียก /api/generate เพื่อสร้างข้อความตอบกลับ"""
    res = requests.post(
        f"{API_URL}/api/generate",
        json={
            "model": MODEL_NAME,
            "prompt": prompt,
            "stream": False
        }
    )
    res.raise_for_status()
    return res.json()["response"]

if __name__ == "__main__":
    print(generate("สวัสดี"))

สวัสดีครับ! ยินดีที่ได้รู้จักครับ ผมชื่อ Typhoon สร้างโดย SCB 10X เพื่อให้ความช่วยเหลืออย่างมีประโยชน์ ปลอดภัย และซื่อสัตย์ครับ มีอะไรให้ผมช่วยไหมครับ?


In [3]:
import os
import re

# === กำหนด path ===
input_dir = 'data'
output_dir = 'preprocessdata'

# === ฟังก์ชัน clean ข้อความ ===
def clean_text(text):
    # ลบ space, tab, newline ซ้ำกัน
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\t+', ' ', text)
    
    # ลบอักขระพิเศษที่ไม่จำเป็น
    text = re.sub(r'[^\w\sก-๙.,()/%&\-–“”"\':]', '', text)

    return text.strip()

# === ตรวจสอบให้มีโฟลเดอร์ปลายทาง ===
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# === วนลูป clean ไฟล์ทั้งหมด ===
for filename in os.listdir(input_dir):
    if filename.endswith('.txt'):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        with open(input_path, 'r', encoding='utf-8') as infile:
            raw_text = infile.read()

        cleaned_text = clean_text(raw_text)

        with open(output_path, 'w', encoding='utf-8') as outfile:
            outfile.write(cleaned_text)

        print(f"[✓] Cleaned: {filename}")

print("\n✅ Cleaning completed and saved to:", output_dir)

[✓] Cleaned: BUSINESS LOOK Day.txt
[✓] Cleaned: ที่อยู่สำนักงาน.txt
[✓] Cleaned: บริการและสินค้า.txt
[✓] Cleaned: พนักงานดีเด่น ประจำปี 2567.txt
[✓] Cleaned: วันหยุดประจำปี พ.ศ. 2568.txt
[✓] Cleaned: เกี่ยวกับ ทีทีที บราเธอร์ส.txt

✅ Cleaning completed and saved to: preprocessdata


In [31]:
import os

def load_and_chunk(directory: str, chunk_size: int = 3000):
    """อ่านไฟล์ .txt ในโฟลเดอร์ แล้วแบ่งเป็นชิ้นละ chunk_size คำ"""
    chunks = []
    for fname in os.listdir(directory):
        if not fname.endswith(".txt"):
            continue
        text = open(os.path.join(directory, fname), encoding="utf-8").read()
        words = text.split()
        for i in range(0, len(words), chunk_size):
            chunk = " ".join(words[i : i + chunk_size])
            # เก็บ metadata เบื้องต้นด้วย
            chunks.append({
                "content": chunk,
                "source": fname
            })
    return chunks

# usage
directory = "preprocessdata"
list_of_chunks = load_and_chunk(directory)
print(f"ได้ทั้งหมด {len(list_of_chunks)} chunks")


ได้ทั้งหมด 6 chunks


In [15]:
from sentence_transformers import SentenceTransformer

# โหลดโมเดล embedding
embed_model = SentenceTransformer("all-mpnet-base-v2")

In [32]:
# แปลงเป็นเวกเตอร์ 768 มิติ
texts = [c["content"] for c in list_of_chunks]
embeddings = embed_model.encode(texts, show_progress_bar=True)

# เพิ่ม embedding ลงใน list_of_chunks
for chunk, emb in zip(list_of_chunks, embeddings):
    chunk["vector"] = emb.tolist()


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.49it/s]


In [33]:
import psycopg2

conn = psycopg2.connect(
    dbname="mydb", user="admin", password="1234",
    host="localhost", port="5432"
)
cur = conn.cursor()
# สร้างตารางถ้ายังไม่มี
cur.execute("""
    CREATE TABLE IF NOT EXISTS documents (
        id SERIAL PRIMARY KEY,
        content TEXT,
        source TEXT,
        embedding VECTOR(768)
    )
""")
conn.commit()

# แทรกข้อมูล
for c in list_of_chunks:
    cur.execute("""
        INSERT INTO documents (content, source, embedding)
        VALUES (%s, %s, %s)
    """, (c["content"], c["source"], c["vector"]))
conn.commit()
cur.close()
conn.close()


In [37]:
import psycopg2
from sklearn.metrics.pairwise import cosine_similarity

# 2. ฟังก์ชันสืบค้นเอกสารจาก Postgres ตามความใกล้เคียงของเวกเตอร์
def retrieve_similar(query: str, top_k: int = 5):
    """
    รับข้อความค้นหา (query) -> แปลงเป็นเวกเตอร์ -> 
    SELECT เอกสารที่ embedding ใกล้เคียงที่สุด top_k ชิ้น
    """
    # แปลง query เป็นเวกเตอร์
    q_vec = embed_model.encode([query], show_progress_bar=False)[0].tolist()

    # เชื่อมต่อฐานข้อมูล
    conn = psycopg2.connect(
        dbname="mydb", user="admin", password="1234",
        host="localhost", port="5432"
    )
    cur = conn.cursor()

    # ใช้ operator <-> ของ pgvector เพื่อคำนวณระยะ Euclidean
    # ต้องแปลง array เป็น vector ด้วย ::vector
    cur.execute("""
        SELECT content, source
        FROM documents
        ORDER BY embedding <-> (%s::vector)
        LIMIT %s
    """, (q_vec, top_k))

    rows = cur.fetchall()
    cur.close()
    conn.close()

    # คืนผลเป็น list ของ dict
    return [{"content": r[0], "source": r[1]} for r in rows]


# 3. ฟังก์ชันรวม context และเรียก LLM (ตัวอย่างสำหรับ local API)
import requests

API_URL = "http://localhost:5433"       # แก้เป็น URL ของ LLM service
MODEL_NAME = "scb10x/typhoon2.1-gemma3-4b:latest"

def generate_rag_response(query: str, top_k: int = 5) -> str:
    # 3.1 สืบค้นเอกสารที่เกี่ยวข้อง
    docs = retrieve_similar(query, top_k)
    
    # ตรวจสอบความเกี่ยวข้องของเอกสารที่พบ
    # หาก similarity score ต่ำเกินไป จะถือว่าไม่พบข้อมูลที่เกี่ยวข้อง
    threshold = 0.3  # ปรับค่าตามความเหมาะสม
    query_vec = embed_model.encode([query], show_progress_bar=False)[0]
    docs_relevant = False
    
    if docs:
        # ตรวจสอบ similarity กับเอกสารที่ได้
        doc_vec = embed_model.encode([docs[0]["content"]], show_progress_bar=False)[0]
        similarity = cosine_similarity([query_vec], [doc_vec])[0][0]
        docs_relevant = similarity > threshold

    if docs_relevant:
        # มีเอกสารที่เกี่ยวข้อง ใช้ RAG
        context = "\n\n".join(
            f"Source: {doc['source']}\n{doc['content']}" for doc in docs
        )
        prompt = (
            "คุณคือผู้ช่วยอัจฉริยะชื่อ TTT-Assistant ของบริษัท TTT Brothers กรุณาตอบคำถามโดยใช้ข้อมูลที่ให้มา\n"
            f"---\nContext:\n{context}\n---\n"
            f"Question: {query}\nAnswer:"
        )
    else:
        # ไม่พบเอกสารที่เกี่ยวข้อง ใช้ LLM ตอบคำถามทั่วไป
        prompt = (
            "คุณคือผู้ช่วยอัจฉริยะชื่อ TTT-Assistant ของบริษัท TTT Brothers "
            "ที่สามารถตอบคำถามได้อย่างสุภาพและเป็นมิตร\n\n"
            f"Question: {query}\nAnswer:"
        )

    # เรียก LLM API
    res = requests.post(
        f"{API_URL}/api/generate",
        json={
            "model": MODEL_NAME,
            "prompt": prompt,
            "stream": False
        }
    )
    res.raise_for_status()
    return res.json()["response"]

# 4. วิธีใช้งาน
if __name__ == "__main__":
    question = "บริษัท TTT Brothers เกี่ยวกับอะไร?"
    answer = generate_rag_response(question, top_k=3)
    print(answer)

สวัสดีครับ ผม Typhoon ผู้ช่วยอัจฉริยะจาก SCB 10X ครับ

TTT Brothers เป็นบริษัทที่เชี่ยวชาญด้านการให้บริการด้านการขนส่งโลจิสติกส์และการจัดการห่วงโซ่อุปทานครบวงจรครับ เราให้บริการหลากหลายรูปแบบ เช่น การขนส่งสินค้าทางบก ทางเรือ ทางอากาศ และบริการคลังสินค้า รวมถึงการจัดการระบบ Supply Chain ที่ครอบคลุม เพื่อให้ลูกค้าได้รับบริการที่มีประสิทธิภาพและตอบโจทย์ความต้องการได้อย่างครบวงจร

หากมีคำถามเพิ่มเติมหรือต้องการข้อมูลเฉพาะด้านอื่นๆ สามารถสอบถามได้เลยนะครับ ยินดีให้บริการครับ
